# Libraries

In [1]:
import sys
import os
import gc
import warnings
import random
from copy import deepcopy
import random
import math

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import torch
from torch import nn
import torch.nn.functional as F
from torch.nn import Parameter
from torch.utils.data import Dataset, TensorDataset, DataLoader
from madgrad import MADGRAD, MirrorMADGRAD
from torch.optim import AdamW
import transformers
from transformers import (
    get_constant_schedule,
    get_constant_schedule_with_warmup,
    get_cosine_schedule_with_warmup,
    get_cosine_with_hard_restarts_schedule_with_warmup,
    get_linear_schedule_with_warmup,
    get_polynomial_decay_schedule_with_warmup
)

from sklearn.model_selection import StratifiedKFold, KFold
    
from tqdm.notebook import tqdm

warnings.filterwarnings("ignore")
tqdm.pandas()

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]


# Load Data

In [2]:
PROJECT_DIR = "."
DATA_DIR = f"{PROJECT_DIR}/data"

In [3]:
train = pd.read_csv(f"{DATA_DIR}/train.csv")
test = pd.read_csv(f"{DATA_DIR}/public_test.csv")
submission = pd.read_csv(f"{DATA_DIR}/sample_submission.csv")

train.head(20)

,corrupted_text,correct_text,category
0,"Считает, что ссожет ить вечно!","Считает, что сможет жить вечно!",phrases
1,20 миллионов и ни ппнни меньше!,20 миллионов и ни пенни меньше!,phrases
2,- Но и мы умрём.,- Но и мы умрём.,phrases
3,Ты вобще молоцд.,Ты вобще молодец.,phrases
4,Вы имеетн в виду силу ьога?,Вы имеете в виду силу бога?,phrases
5,"Я поливала вцеты, я обещала Кауфманам их пaлиать.","Я поливала цветы, я обещала Кауфманам их полив...",phrases
6,"Не обращайте на меня винмнаия, м-р Дженнингс?","Не обращайте на меня внимания, м-р Дженнингс?",phrases
7,босаснов зефир ванильный темной глазури,БоссаНова зефир ванильный темной глазури,products
8,А ночью Джоуи Триббиани!,А ночью Джоуи Триббиани!,phrases
9,Филир Морис комп Эксперт,Филип Морис комп Эксперт,products


In [4]:
test

,corrupted_text
0,"Скажи сыну, тчо всё будет хорош, Гордон."
1,"Лаюно. Отсавайся, сколько хочешь."
2,Почему вы здесь? Прошлой очью я слышаш выстрелы.
3,А какая бла у Джорджи работа в последний дееь?
4,Уош мне родря!
...,...
185048,Мылр горчичное Ивановское
185049,"Хорошо получилось, добрыи сэ. Вы победитель."
185050,Только чтa закончили.
185051,Кого? — Про ког он?


In [5]:
train.category.value_counts()

phrases     1088214
products     369523
jokes         13957
books          8726
Name: category, dtype: int64

# Правильные слова

In [ ]:
def preprocessor(text):
    return "".join([char for char in text if char.isalnum() or char == " "])

In [ ]:
correct_words = set()

for el in tqdm(train["correct_text"].apply(preprocessor)):
    correct_words.update(set(el.split()))

len(correct_words)

# Пайплайн замены

In [14]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModel, BertTokenizerFast, BertForMaskedLM

tokenizer = BertTokenizerFast.from_pretrained("sberbank-ai/ruBert-large")
model = BertForMaskedLM.from_pretrained("sberbank-ai/ruBert-large")

pipe = pipeline(task="fill-mask", model=model, tokenizer=tokenizer, device=torch.device("cuda"))

Some weights of the model checkpoint at sberbank-ai/ruBert-large were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RuntimeError: No CUDA GPUs are available